<a href="https://colab.research.google.com/github/tessyworld/tessy/blob/main/code_bot_complex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import logging
import os
import sqlite3

from telegram import InlineKeyboardButton, InlineKeyboardMarkup, ParseMode
from telegram.ext import CommandHandler, ConversationHandler, Filters, MessageHandler, Updater

# Configura il logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

# Costanti per il ConversationHandler
GENDER, AGE, EMAIL, END = range(4)

# Imposta il token del tuo bot su Telegram
TOKEN = os.environ.get('BOT_TOKEN')

# Crea un'istanza del bot
bot = telegram.Bot(token=TOKEN)

# Funzione per gestire il comando /start
def start(update, context):
    user_id = update.message.from_user.id
    first_name = update.message.from_user.first_name
    last_name = update.message.from_user.last_name
    username = update.message.from_user.username

    # Salva le informazioni dell'utente nel database
    conn = sqlite3.connect('users.db')
    c = conn.cursor()
    c.execute("INSERT INTO users (user_id, first_name, last_name, username) VALUES (?, ?, ?, ?)",
              (user_id, first_name, last_name, username))
    conn.commit()
    conn.close()

    context.bot.send_message(chat_id=update.message.chat_id, text="Ciao! Benvenuto nel mio bot.")

# Funzione per gestire il comando /help
def help(update, context):
    context.bot.send_message(chat_id=update.message.chat_id, text="Ecco la lista dei comandi disponibili:\n\n"
                                                                "/start - Inizia a usare il bot\n"
                                                                "/profile - Mostra il tuo profilo\n"
                                                                "/register - Registra il tuo account")

# Funzione per gestire il comando /profile
def profile(update, context):
    user_id = update.message.from_user.id

    # Cerca le informazioni dell'utente nel database
    conn = sqlite3.connect('users.db')
    c = conn.cursor()
    c.execute("SELECT * FROM users WHERE user_id=?", (user_id,))
    user = c.fetchone()
    conn.close()

    if user:
        text = f"<b>Profilo di {user[1]}</b>\n\n" \
               f"<b>ID utente:</b> {user[0]}\n" \
               f"<b>Nome:</b> {user[1]}\n" \
               f"<b>Cognome:</b> {user[2]}\n" \
               f"<b>Username:</b> @{user[3]}"
    else:
        text = "Non hai ancora registrato il tuo account. Usa il comando /register per farlo."

    context.bot.send_message(chat_id=update.message.chat_id, text=text, parse_mode=ParseMode.HTML)

# Funzione per gestire il comando /register
def register_start(update, context):
    context.bot.send_message(chat_id=update.message.chat_id,
                             text="Per registrare il tuo account, per favore rispondi a queste domande:\n\n"
                                  "Qual è il tuo genere? (M/F/Altro)")

    return GENDER

def register_gender(update, context):
    user_id = update.message.from_user.id
    gender = update.message.text

    # Salva il genere dell'utente nel database
